# 00 - Data Split

## Purpose
Load the raw housing dataset and split it by **date** (temporal split) to prevent data leakage.

### Why Temporal Split?
- Random split causes data leakage with time series data
- Future data would leak into training set
- Model would appear to perform better than reality

### Split Strategy
- **Training**: 2012-2019 (used to train the model)
- **Evaluation**: 2020-2021 (used for validation and hyperparameter tuning)
- **Hold-out**: 2022-2023 (completely untouched until final predictions)

In [1]:
import pandas as pd
from pathlib import Path

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 1. Load Raw Data

In [2]:
# Define paths
DATA_DIR = Path("../data/raw")
ORIGINAL_FILE = DATA_DIR / "untouched_raw_original.csv"

print(f"Loading data from: {ORIGINAL_FILE}")
df = pd.read_csv(ORIGINAL_FILE)
print(f"\nDataset shape: {df.shape[0]:,} rows x {df.shape[1]} columns")

Loading data from: ..\data\raw\untouched_raw_original.csv

Dataset shape: 884,092 rows x 39 columns


In [3]:
# Preview the data
df.head(2)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
0,2012-03-31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,0.943662,0.142857,0.043478,ATL,30002,2012,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Alpharetta
1,2012-04-30,61870.0,245000.0,40.723982,130.528256,22.0,29.0,56.0,69.0,89.5,0.946642,0.090909,0.034483,ATL,30002,2012,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202421.064584,Atlanta-Sandy Springs-Alpharetta


In [4]:
# Check column names
print("Columns:")
print(df.columns.tolist())

Columns:
['date', 'median_sale_price', 'median_list_price', 'median_ppsf', 'median_list_ppsf', 'homes_sold', 'pending_sales', 'new_listings', 'inventory', 'median_dom', 'avg_sale_to_list', 'sold_above_list', 'off_market_in_two_weeks', 'city', 'zipcode', 'year', 'bank', 'bus', 'hospital', 'mall', 'park', 'restaurant', 'school', 'station', 'supermarket', 'Total Population', 'Median Age', 'Per Capita Income', 'Total Families Below Poverty', 'Total Housing Units', 'Median Rent', 'Median Home Value', 'Total Labor Force', 'Unemployed Population', 'Total School Age Population', 'Total School Enrollment', 'Median Commute Time', 'price', 'city_full']


## 2. Extract Year from Date Column

In [5]:
# Convert date column to datetime and extract year
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year

# Check year range
print(f"Year range: {df['year'].min()} to {df['year'].max()}")

Year range: 2012 to 2023


In [6]:
# Year distribution
print("\nRows per year:")
print(df['year'].value_counts().sort_index())


Rows per year:
year
2012    62260
2013    74712
2014    74712
2015    74712
2016    74712
2017    74712
2018    74712
2019    74712
2020    74712
2021    74712
2022    74712
2023    74712
Name: count, dtype: int64


## 3. Temporal Split (by Date)

We split by date instead of randomly to prevent data leakage:
- **Training**: 2012-2019
- **Evaluation**: 2020-2021  
- **Hold-out**: 2022-2023

In [7]:
# Define cutoff years
EVAL_CUTOFF = 2020
HOLDOUT_CUTOFF = 2022

# Split the data
train_df = df[df['year'] < EVAL_CUTOFF].copy()
eval_df = df[(df['year'] >= EVAL_CUTOFF) & (df['year'] < HOLDOUT_CUTOFF)].copy()
holdout_df = df[df['year'] >= HOLDOUT_CUTOFF].copy()

print("="*50)
print("TEMPORAL DATA SPLIT SUMMARY")
print("="*50)
print(f"Training set (2012-2019):   {len(train_df):,} rows")
print(f"Evaluation set (2020-2021): {len(eval_df):,} rows")
print(f"Hold-out set (2022-2023):   {len(holdout_df):,} rows")
print("="*50)

TEMPORAL DATA SPLIT SUMMARY
Training set (2012-2019):   585,244 rows
Evaluation set (2020-2021): 149,424 rows
Hold-out set (2022-2023):   149,424 rows


## 4. Verify Split

In [8]:
# Verify year ranges
print(f"Training years: {train_df['year'].min()} - {train_df['year'].max()}")
print(f"Evaluation years: {eval_df['year'].min()} - {eval_df['year'].max()}")
print(f"Hold-out years: {holdout_df['year'].min()} - {holdout_df['year'].max()}")

Training years: 2012 - 2019
Evaluation years: 2020 - 2021
Hold-out years: 2022 - 2023


## 5. Save Split Datasets

In [9]:
# Save to CSV
train_df.to_csv(DATA_DIR / "train.csv", index=False)
eval_df.to_csv(DATA_DIR / "eval.csv", index=False)
holdout_df.to_csv(DATA_DIR / "holdout.csv", index=False)

print("✅ Saved datasets:")
print(f"  - {DATA_DIR / 'train.csv'}")
print(f"  - {DATA_DIR / 'eval.csv'}")
print(f"  - {DATA_DIR / 'holdout.csv'}")

✅ Saved datasets:
  - ..\data\raw\train.csv
  - ..\data\raw\eval.csv
  - ..\data\raw\holdout.csv


In [10]:
print("\n✅ Data splitting complete! Proceed to notebook 01 for EDA and cleaning.")


✅ Data splitting complete! Proceed to notebook 01 for EDA and cleaning.
